# Objectives
After completing this lab, you will be able to:

- Use Amazon SageMaker to create a hyperparameter training job

- Tune an XGBoost model by using Amazon SageMaker

- Test the tuned model by using performance metrics

## Información general
Este laboratorio es una continuación de los laboratorios guiados del Módulo 3.

En este laboratorio, creará un trabajo de ajuste de hiperparámetros para ajustar el modelo que creó anteriormente. A continuación, comparará las métricas de los dos modelos.

## Presentación de la situación empresarial
Trabaja para un proveedor de atención médica y desea mejorar la detección de anomalías en los pacientes de ortopedia.

Se le asignó la tarea de resolver este problema con el uso de machine learning (ML). Tiene acceso a un conjunto de datos que contiene seis funciones biomecánicas y un objetivo de normal o anormal. Puede usar este conjunto de datos para entrenar un modelo de ML para predecir si un paciente tendrá una anomalía.

## Acerca de este conjunto de datos
Este conjunto de datos biomédicos fue creado por el Dr. Henrique da Mota durante un periodo de residencia médica en el Group of Applied Research in Orthopaedics (GARO) del Centre Médico-Chirurgical de Réadaptation des Massues, Lyon, Francia. Los datos se organizaron en dos tareas de clasificación diferentes pero que se relacionan.

La primera tarea consiste en clasificar pacientes como pertenecientes a una de tres categorías:

Normal (100 pacientes)
Hernia de disco (60 pacientes)
Espondilolistesis (150 pacientes)
Para la segunda tarea, las categorías hernia de disco y espondilolistesis se fusionaron en una sola categoría que se denomina anormal. Por ende, la segunda tarea consiste en clasificar a los pacientes como pertenecientes a una de dos categorías: normal (100 pacientes) o anormal (210 pacientes).

## Información de atributos
Cada paciente está representado en el conjunto de datos por seis atributos biomecánicos que se derivan de la forma y la orientación de la pelvis y la columna lumbar (en este orden):

Pelvic incidence (Incidencia pélvica)
Pelvic tilt (Inclinación pélvica)
Lumbar lordosis angle (Ángulo de lordosis lumbar)
Sacral slope (Pendiente sacra)
Pelvic radius (Radio pélvico)
Grade of spondylolisthesis (Grado de espondilolistesis)
Se usa la siguiente convención para las etiquetas de clase:

DH (hernia de disco)
SL (espondilolistesis)
NO (normal)
AB (anormal)
Para obtener más información sobre este conjunto de datos, consulte la página web del conjunto de datos de la columna vertebral.

## Atribuciones del conjunto de datos
Este conjunto de datos se obtuvo de: Dua, D. y Graff, C. (2019). Repositorio de Machine Learning de UCI (http://archive.ics.uci.edu/ml). Irvine, CA: Universidad de California, School of Information and Computer Science.

# Configuración del laboratorio

Debido a que esta solución se divide en varios laboratorios en el módulo, debe ejecutar las siguientes celdas para poder cargar los datos y entrenar el modelo que se implementará.

**Nota:** La configuración puede tardar hasta 5 minutos en completarse.

## Importación de los datos y entrenamiento, prueba y validación del modelo

Ejecute las siguientes celdas para importar los datos, entrenar, probar y validar el modelo y dejarlo listo para utilizarse. 

**Nota:** Las siguientes celdas representan los pasos clave de los laboratorios anteriores.

Para ajustar el modelo, debe estar listo y, luego, se puede afinar con hiperparámetros en el paso 2.

# Desde tu terminal (PowerShell o CMD, no en el notebook) crea y activa el entorno con Python 3.12:
    
py -3.12 -m venv venv

.\venv\Scripts\activate   # PowerShell

pip install requests pandas scipy boto3 sagemaker scikit-learn seaborn matplotlib ipykernel

python -m ipykernel install --user --name=venv --display-name "Python 3.12 (venv)"


In [1]:
import time
start = time.time()
import warnings, requests, zipfile, io
warnings.simplefilter('ignore')
import pandas as pd
from scipy.io import arff

import os
import boto3
import sagemaker
from sagemaker.image_uris import retrieve
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Rodrigo\AppData\Local\sagemaker\sagemaker\config.yaml


**Nota:** La siguiente celda tarda aproximadamente **10** minutos en completarse. Observe el código y cómo se procesa, esto le permite entender mejor lo que sucede en segundo plano. Tenga presente que esta celda completa todos los pasos que hizo en laboratorios anteriores de este módulo, entre ellos:
 - Importación de los datos
 - Carga de los datos en un DataFrame
 - División de los datos en conjuntos de datos de entrenamiento, prueba y validación
 - Carga de los conjuntos de datos divididos en S3
 - Entrenamiento, prueba y validación del modelo con los conjuntos de datos

pip install awscli
aws configure
C:\Users\Rodrigo\.aws

[default]
aws_access_key_id=aws_access_key_id
aws_secret_access_key=aws_secret_access_key
aws_session_token=aws_session_token

In [3]:
bucket='c174660a4519473l11469662t1w975050097455-labbucket-shf2otxd36wy'

In [5]:
%%time

def plot_roc(test_labels, target_predicted_binary):
    TN, FP, FN, TP = confusion_matrix(test_labels, target_predicted_binary).ravel()
    # Sensitivity, hit rate, recall, or true positive rate
    Sensitivity  = float(TP)/(TP+FN)*100
    # Specificity or true negative rate
    Specificity  = float(TN)/(TN+FP)*100
    # Precision or positive predictive value
    Precision = float(TP)/(TP+FP)*100
    # Negative predictive value
    NPV = float(TN)/(TN+FN)*100
    # Fall out or false positive rate
    FPR = float(FP)/(FP+TN)*100
    # False negative rate
    FNR = float(FN)/(TP+FN)*100
    # False discovery rate
    FDR = float(FP)/(TP+FP)*100
    # Overall accuracy
    ACC = float(TP+TN)/(TP+FP+FN+TN)*100

    print(f"Sensitivity or TPR: {Sensitivity}%")    
    print(f"Specificity or TNR: {Specificity}%") 
    print(f"Precision: {Precision}%")   
    print(f"Negative Predictive Value: {NPV}%")  
    print( f"False Positive Rate: {FPR}%") 
    print(f"False Negative Rate: {FNR}%")  
    print(f"False Discovery Rate: {FDR}%" )
    print(f"Accuracy: {ACC}%") 

    test_labels = test.iloc[:,0];
    print("Validation AUC", roc_auc_score(test_labels, target_predicted_binary) )

    fpr, tpr, thresholds = roc_curve(test_labels, target_predicted_binary)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % (roc_auc))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")

    # create the axis of thresholds (scores)
    ax2 = plt.gca().twinx()
    ax2.plot(fpr, thresholds, markeredgecolor='r',linestyle='dashed', color='r')
    ax2.set_ylabel('Threshold',color='r')
    ax2.set_ylim([thresholds[-1],thresholds[0]])
    ax2.set_xlim([fpr[0],fpr[-1]])

    print(plt.figure())

def plot_confusion_matrix(test_labels, target_predicted):
    matrix = confusion_matrix(test_labels, target_predicted)
    df_confusion = pd.DataFrame(matrix)
    colormap = sns.color_palette("BrBG", 10)
    sns.heatmap(df_confusion, annot=True, fmt='.2f', cbar=None, cmap=colormap)
    plt.title("Confusion Matrix")
    plt.tight_layout()
    plt.ylabel("True Class")
    plt.xlabel("Predicted Class")
    plt.show()
    

f_zip = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip'
r = requests.get(f_zip, stream=True)
Vertebral_zip = zipfile.ZipFile(io.BytesIO(r.content))
Vertebral_zip.extractall()

data = arff.loadarff('column_2C_weka.arff')
df = pd.DataFrame(data[0])

class_mapper = {b'Abnormal':1,b'Normal':0}
df['class']=df['class'].replace(class_mapper)

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

train, test_and_validate = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])
test, validate = train_test_split(test_and_validate, test_size=0.5, random_state=42, stratify=test_and_validate['class'])

prefix='lab3'

train_file='vertebral_train.csv'
test_file='vertebral_test.csv'
validate_file='vertebral_validate.csv'

s3_resource = boto3.Session().resource('s3')
def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

upload_s3_csv(train_file, 'train', train)
upload_s3_csv(test_file, 'test', test)
upload_s3_csv(validate_file, 'validate', validate)

container = retrieve('xgboost',boto3.Session().region_name,'1.0-1')

hyperparams={"num_round":"42",
             "eval_metric": "auc",
             "objective": "binary:logistic",
             "silent" : 1}

s3_output_location="s3://{}/{}/output/".format(bucket,prefix)
xgb_model=sagemaker.estimator.Estimator(container,
                                       sagemaker.get_execution_role(),
                                       instance_count=1,
                                       instance_type='ml.m5.2xlarge',
                                       output_path=s3_output_location,
                                        hyperparameters=hyperparams,
                                        sagemaker_session=sagemaker.Session())

train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/".format(bucket,prefix,train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/".format(bucket,prefix,validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

xgb_model.fit(inputs=data_channels, logs=False)

batch_X = test.iloc[:,1:];

batch_X_file='batch-in.csv'
upload_s3_csv(batch_X_file, 'batch-in', batch_X)

batch_output = "s3://{}/{}/batch-out/".format(bucket,prefix)
batch_input = "s3://{}/{}/batch-in/{}".format(bucket,prefix,batch_X_file)

xgb_transformer = xgb_model.transformer(instance_count=1,
                                       instance_type='ml.m5.2xlarge',
                                       strategy='MultiRecord',
                                       assemble_with='Line',
                                       output_path=batch_output)

xgb_transformer.transform(data=batch_input,
                         data_type='S3Prefix',
                         content_type='text/csv',
                         split_type='Line')
xgb_transformer.wait(logs=False)

CPU times: total: 719 ms
Wall time: 1.85 s


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 get_ipython().run_cell_magic('time', '', '\ndef plot_roc(test_labels, target_predicted_b     │
│   2                                                                                              │
│                                                                                                  │
│ c:\Projects\inacap\venv\Lib\site-packages\IPython\core\interactiveshell.py:2565 in               │
│ run_cell_magic                                                                                   │
│                                                                                                  │
│   2562 │   │   │                                                                                 │
│   2563 │   │   │   with self.builtin_trap:                                                       │
│   2564 │   │   │   │   args = (magic_arg_s, cell)                                                │
│ ❱ 2565 │   │   │   │   result = fn(*args, **kwargs)                                              │
│   2566 │   │   │                                                                                 │
│   2567 │   │   │   # The code below prevents the output from being displayed                     │
│   2568 │   │   │   # when using magics with decorator @output_can_be_silenced                    │
│                                                                                                  │
│ c:\Projects\inacap\venv\Lib\site-packages\IPython\core\magics\execution.py:1470 in time          │
│                                                                                                  │
│   1467 │   │   │   print(f"Parser   : {_format_time(tp)}")                                       │
│   1468 │   │   if interrupt_occured:                                                             │
│   1469 │   │   │   if exit_on_interrupt and captured_exception:                                  │
│ ❱ 1470 │   │   │   │   raise captured_exception                                                  │
│   1471 │   │   │   return                                                                        │
│   1472 │   │   return out                                                                        │
│   1473                                                                                           │
│                                                                                                  │
│ c:\Projects\inacap\venv\Lib\site-packages\IPython\core\magics\execution.py:1434 in time          │
│                                                                                                  │
│   1431 │   │   else:                                                                             │
│   1432 │   │   │   st = clock2()                                                                 │
│   1433 │   │   │   try:                                                                          │
│ ❱ 1434 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1435 │   │   │   │   out = None                                                                │
│   1436 │   │   │   │   # multi-line %%time case                                                  │
│   1437 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>:96                                                                                   │
│ in upload_s3_csv:94                                                                              │
│                                                                                                  │
│ c:\Projects\inacap\venv\Lib\site-packages\boto3\resources\f